In [135]:
import pandas as pd


In [136]:
gerentes_df = pd.read_excel('../cleaned_dataset_no_percentile_outliers.xlsx')
metrics_df = pd.read_excel('metrics.xlsx')

In [137]:
# Converter a coluna de categorias em uma string para cada gerente
labels = gerentes_df['metrics_category'].fillna('').astype(str)

In [138]:
# Substituir valores NaN por string vazia para evitar erros
gerentes_df['agile_methods'] = gerentes_df['agile_methods'].fillna('')

# Dividir a coluna de métodos ágeis para cada método se tornar um termo único
gerentes_df['agile_methods_split'] = gerentes_df['agile_methods'].apply(lambda x: x.split(', '))

In [ ]:
# Explorar os métodos ágeis únicos no dataset
unique_methods = set()
gerentes_df['agile_methods_split'].apply(unique_methods.update)


# Outlier detection em metricas ágeis

In [140]:
from collections import Counter


In [ ]:
# Contar a frequência de cada método ágil
all_methods = [method for methods in gerentes_df['agile_methods_split'] for method in methods]
methods_counter = Counter(all_methods)

# Criar um DataFrame a partir do contador de métodos
methods_freq_df = pd.DataFrame(methods_counter.items(), columns=['Method', 'Frequency']).sort_values(by='Frequency', ascending=False)

# Mostrar as 5 primeiras linhas
methods_freq_df.head()

In [142]:
import matplotlib.pyplot as plt


In [ ]:
methods_freq_sorted = methods_freq_df.sort_values(by='Frequency', ascending=False)

# Extrair os valores para o eixo x (métodos) e y (frequência)
methods = methods_freq_sorted['Method']
frequencies = methods_freq_sorted['Frequency']

# Plotar a distribuição de frequência
plt.figure(figsize=(12, 6))
plt.plot(range(len(methods)), frequencies, marker='o', linestyle='-', color='blue')
plt.xticks(range(len(methods)), methods, rotation=90)
plt.xlabel('Agile Methods')
plt.ylabel('Frequency')
plt.title('Normal Curve Style Frequency Distribution of Agile Methods')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [144]:
# --------------------- Remoção por Desvio Padrão ---------------------

mean_frequency = methods_freq_df['Frequency'].mean()


In [ ]:
# Identificar os métodos ágeis cuja frequência é menor que 2
outliers = methods_freq_df[methods_freq_df['Frequency'] < 2]

# Mostrar os métodos identificados como outliers
print("Métodos Ágeis Identificados como Outliers (Frequência < 2):")
print(outliers)

In [ ]:
# Obter os métodos considerados outliers
outliers_methods = set(outliers['Method'])

# Remover os métodos ágeis que são outliers da coluna 'agile_methods_split'
gerentes_df['agile_methods_filtered'] = gerentes_df['agile_methods_split'].apply(lambda x: [method for method in x if method not in outliers_methods])

# Mostrar o dataset atualizado com a coluna de métodos filtrados
gerentes_df[['agile_methods', 'agile_methods_filtered']].head()

In [ ]:
# Criar um DataFrame para visualizar os métodos que foram removidos como outliers
removed_methods_df = pd.DataFrame(outliers, columns=['Method', 'Frequency'])

# Mostrar os métodos removidos
print("Métodos Ágeis Removidos do Dataset (Outliers):")
print(removed_methods_df)

In [148]:
gerentes_df.to_excel('filtered_dataset_metrics.xlsx')

# Treinamento

In [155]:
filtered_data = pd.read_excel('filtered_data_3.xlsx')

In [ ]:
filtered_data['metrics_category'].fillna('')

In [161]:
filtered_data['combined_features'] = (
    filtered_data['role'].fillna('') + ' ' +
    filtered_data['years_exp'].fillna('') + ' ' +
    filtered_data['org_size'].fillna('') + ' ' +
    filtered_data['org_field'].fillna('') + ' ' +
    filtered_data['agile_methods_filtered'].str.join('') + ' ' +
    filtered_data['use_metrics_planning'].fillna('Não') + ' ' +
    filtered_data['use_metrics_review'].fillna('Não') + ' ' +
    filtered_data['use_metrics_weekly'].fillna('Não') + ' ' +
    filtered_data['use_metrics_daily'].fillna('Não') + ' ' +
    filtered_data['use_metrics_retro'].fillna('Não')
)


In [162]:
from sklearn.model_selection import train_test_split

# Definir as features (X) e o alvo (y)
X = filtered_data['combined_features']
y = filtered_data['metrics_category']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [168]:
X_train = X_train.fillna('')
X_test = X_test.fillna('')

In [170]:
from sklearn.naive_bayes import MultinomialNB

# Inicializar o classificador Naive Bayes
classifier = MultinomialNB(alpha=1.0)  # Ajuste de alpha para suavização


# Treinar o modelo
classifier.fit(X_train_tfidf, y_train)

# Fazer previsões no conjunto de teste
y_pred = classifier.predict(X_test_tfidf)

In [174]:
from sklearn.metrics import classification_report

# Imprimir o relatório de classificação
report = classification_report(y_test, y_pred)
print(report)

                                                                         precision    recall  f1-score   support

                                                 Cronograma e progresso       0.00      0.00      0.00         1
                                        Cronograma e progresso, Pessoas       0.00      0.00      0.00         1
                              Cronograma e progresso, Pessoas, Processo       0.00      0.00      0.00         1
                     Cronograma e progresso, Pessoas, Processo, Cliente       0.00      0.00      0.00         1
                               Cronograma e progresso, Pessoas, Produto       0.00      0.00      0.00         2
                     Cronograma e progresso, Pessoas, Produto, Processo       0.00      0.00      0.00         2
         Cronograma e progresso, Pessoas, Produto, Processo, Tecnologia       0.00      0.00      0.00         2
Cronograma e progresso, Pessoas, Produto, Processo, Tecnologia, Cliente       0.03      0.50   

/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 